In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [8]:
len(pt_features[pt_features['isCase']==False])

10975

In [9]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,non_insomnia_GP_consultations,stroke,non_stroke_vascular_disease,hypertension,diabetes,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,insomnia_count:0,insomnia_count:above_10,non_insomnia_GP_consultations:0,insomnia_count:1_5,insomnia,insomnia_count:6_10,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,other_sedatives_100_pdds,antipsychotics_100_pdds,depot_antipsychotics_100_pdds,antidepressants_100_pdds,benzo_and_z_drugs_any,insomnia_any,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,antidepressants_100_pdds:0,antidepressants_100_pdds:1_10,antidepressants_100_pdds:11_100,antidepressants_100_pdds:101_1000,antidepressants_100_pdds:1001_10000,antidepressants_100_pdds:above_10000,antipsychotics_100_pdds:0,antipsychotics_100_pdds:1_10,antipsychotics_100_pdds:11_100,antipsychotics_100_pdds:101_1000,antipsychotics_100_pdds:1001_10000,antipsychotics_100_pdds:above_10000,depot_antipsychotics_100_pdds:0,depot_antipsychotics_100_pdds:1_10,depot_antipsychotics_100_pdds:11_100,depot_antipsychotics_100_pdds:101_1000,depot_antipsychotics_100_pdds:1001_10000,depot_antipsychotics_100_pdds:above_10000,other_sedatives_100_pdds:0,other_sedatives_100_pdds:1_10,other_sedatives_100_pdds:11_100,other_sedatives_100_pdds:101_1000,other_sedatives_100_pdds:1001_10000,other_sedatives_100_pdds:above_10000,benzo_and_z_drugs_100_pdds:0,benzo_and_z_drugs_100_pdds:1_10,benzo_and_z_drugs_100_pdds:11_100,benzo_and_z_drugs_100_pdds:101_1000,benzo_and_z_drugs_100_pdds:1001_10000,benzo_and_z_drugs_100_pdds:above_10000,mood_stabilisers_100_pdds:0,mood_stabilisers_100_pdds:1_10,mood_stabilisers_100_pdds:11_100,mood_stabilisers_100_pdds:101_1000,mood_stabilisers_100_pdds:1001_10000,mood_stabilisers_100_pdds:above_10000
12722,16423058,22,58,0,2004-08-31,True,6578.0,1996-04-26,2009-01-19,8086,82,85,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.000000,0.000000,0.000000,8.774765,0.0,4.079857,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
18644,4668024,26,24,1,2007-01-23,True,1917.0,1997-10-31,2012-01-16,3814,81,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
18653,14314319,20,319,1,2005-11-30,True,4693.0,1995-10-06,2009-12-23,48217,85,105,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,1.026221,10.877731,10.949086,0.000000,0.0,26.004824,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8144,1122511,32,511,1,2009-04-25,False,NaN,2001-04-06,2013-02-27,37906,77,40,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5034,101068,26,68,1,2010-01-25,True,6578.0,1999-10-05,2011-02-28,8878,84,172,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.179493,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.

In [10]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'non_insomnia_GP_consultations', 'stroke',
       'non_stroke_vascular_disease', 'hypertension', 'diabetes',
       'mental_illness_non_smi', 'mental_illness_smi', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'age_at_index_date:65-69',
       'age_at_index_date:70-74', 'age_at_index_date:75-79',
       'age_at_index_date:80-84', 'age_at_index_date:85-89',
       'age_at_index_date:90-99', 'age_at_index_date:above_99',
       'insomnia_count:0', 'insomnia_count:above_10',
       'non_insomnia_GP_consultations:0', 'insomnia_count:1_5', 'insomnia',
       'insomnia_count:6_10', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'other_sedatives_100_pdds',
       'antipsychotics_100_pdds', 'depot_antipsychotics_100_pdds',
       'antidepressants_100_pdds', 'benzo_and_z_drugs_any', 'insomnia_any

In [11]:
characteristics = [column for column in pt_features.columns if column not in [
    'patid',  'yob', 'pracid', 'index_date', 'isCase',
    'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [12]:
baseline_characteristics = pd.DataFrame(index=characteristics,columns=['Cases','Controls','P value'])

In [13]:
baseline_dichot,baseline_contin = add_baseline_characteristics(baseline_characteristics,characteristics,pt_features)

In [14]:
# baseline_contin

In [15]:
baseline_dichot

,Cases,Controls,P value
age_at_index_date:65-69,440 (4.0%),440 (4.0%),1.000
age_at_index_date:70-74,1038 (9.5%),1038 (9.5%),1.000
age_at_index_date:75-79,2140 (19.5%),2140 (19.5%),1.000
age_at_index_date:80-84,3055 (27.8%),3055 (27.8%),1.000
age_at_index_date:85-89,2832 (25.8%),2832 (25.8%),1.000
age_at_index_date:90-99,1187 (10.8%),1187 (10.8%),1.000
age_at_index_date:above_99,19 (0.2%),19 (0.2%),1.000
antidepressants_100_pdds:0,9298 (84.7%),9801 (89.3%),0.000
antidepressants_100_pdds:1001_10000,366 (3.3%),222 (2.0%),0.000
antidepressants_100_pdds:101_1000,670 (6.1%),463 (4.2%),0.000
